## Drop Athena DB and delete S3 bucket

In [12]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

/home/sagemaker-user/.conda/envs/data_science_on_aws/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### List existing S3 buckets

In [13]:
!aws s3 ls

2024-02-18 20:23:31 211125778552personalizepocvod
2024-02-18 01:27:18 aws-athena-query-results-211125778552-us-east-1
2024-04-13 17:52:38 aws-athena-query-results-us-east-1-211125778552
2024-03-27 00:35:44 aws-glue-assets-211125778552-us-east-1
2024-03-27 00:28:12 aws-glue-assets-211125778552-us-east-2
2024-02-15 21:19:44 sagemaker-studio-12jvao34qlkn
2024-02-15 22:38:05 sagemaker-studio-211125778552-3pjkfc2ijfr
2024-02-17 02:02:09 sagemaker-studio-211125778552-4dcj21sopi3
2024-02-19 03:02:29 sagemaker-studio-211125778552-4rfwbx1bibn
2024-02-15 20:23:46 sagemaker-studio-211125778552-4yhhjbuzjdq
2024-02-17 02:02:35 sagemaker-studio-211125778552-8xxlet4bnrv
2024-02-17 02:02:08 sagemaker-studio-211125778552-rfcwvtinree
2024-02-20 00:38:45 sagemaker-studio-211125778552-yu1t8p5304s
2024-03-21 19:20:47 sagemaker-studio-uyd2sz3oy3
2024-03-08 03:11:27 sagemaker-team11-stanford-dogs
2024-04-13 20:37:15 sagemaker-team4-bucket
2024-03-18 01:49:41 sagemaker-team6-distracted-drivers
2024-02-15 20:2

### Drop the Athena database

In [14]:
from pyathena import connect
import pandas as pd

In [15]:
database_name = "team4"
bucket = "team-4-project-data"

In [16]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [17]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [18]:
statement = "DROP DATABASE IF EXISTS {} CASCADE".format(database_name)
print(statement)

DROP DATABASE IF EXISTS team4 CASCADE


In [19]:
pd.read_sql(statement, conn)

""


In [20]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,sagemaker_featurestore
3,team-8-fec-db
4,team3-court-data


### Remove the Ryanair customer feedback file from the S3 bucket

In [21]:
!aws s3 rm s3://team-4-project-data/ryanair-data/ryanair_reviews.csv

delete: s3://team-4-project-data/ryanair-data/ryanair_reviews.csv


### Remove the S3 bucket

In [22]:
!aws s3 rb s3://team-4-project-data --force

delete: s3://team-4-project-data/athena/staging/f3936029-10e3-473b-b35b-c80ac79767c6.txt
delete: s3://team-4-project-data/athena/staging/544c22a3-7093-4cf9-a8c4-261b609e5bdb.txt
delete: s3://team-4-project-data/athena/staging/f3936029-10e3-473b-b35b-c80ac79767c6.txt.metadata
remove_bucket: team-4-project-data
